Day1-Deal with Missing Value

### In this notebook

 - checking missing value
 - understanding the reason of missing: not exist or not recorded
 - imputing missing value

Read Module

In [2]:
import pandas as pd
import numpy as np

import data

In [3]:
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("Building_Permits.csv")

C:\Users\yuanq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\yuanq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# look at a few rows of the nfl_data file. I can see a handful of missing data already!
nfl_data.sample(5)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
229880,2014-09-14,2014091408,16,4,NaN,09:01,10,541.0,8.0,SEA,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,2014
406958,2017-12-31,2017123104,3,1,1.0,08:54,9,3234.0,0.0,BUF,...,0.091193,0.371447,0.628553,0.362528,0.637472,0.371447,-0.008919,-0.012963,0.004044,2017
4089,2009-09-20,2009092003,14,3,3.0,09:04,10,1444.0,19.0,JAC,...,NaN,0.277623,0.722377,0.310775,0.689225,0.277623,0.033153,NaN,NaN,2009
264192,2014-12-14,2014121410,14,3,4.0,04:38,5,1178.0,5.0,DEN,...,NaN,0.357414,0.642586,0.263619,0.736381,0.357414,-0.093794,NaN,NaN,2014
166072,2012-12-02,2012120200,14,3,2.0,02:30,3,1050.0,40.0,SEA,...,NaN,0.649414,0.350586,0.702654,0.297346,0.350586,-0.053240,NaN,NaN,2012


In [5]:
sf_permits.sample(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
53769,201407030497,8,otc alterations permit,07/03/2014,2752,024,331,NaN,Collingwood,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94114.0,"(37.757074107906604, -122.43543705654334)",1347915143379
92645,201507201954,8,otc alterations permit,07/20/2015,2916,033A,177,NaN,Idora,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,7.0,West of Twin Peaks,94127.0,"(37.744097887591444, -122.45860200757107)",1389042146984
96518,201508245117,8,otc alterations permit,08/24/2015,3596,103,210,NaN,Lexington,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,9.0,Mission,94110.0,"(37.75993614759934, -122.42076235020673)",1393200458203
80555,201503302258,8,otc alterations permit,03/30/2015,6518,030,3334,NaN,25th,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,9.0,Mission,94110.0,"(37.75090026575316, -122.41691172089932)",1376068205974
38611,201402078028,8,otc alterations permit,02/07/2014,5428F,022,2685,NaN,Newhall,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,10.0,Bayview Hunters Point,94124.0,"(37.72863261329471, -122.3982358219582)",1332081445323


### See how many missing data points we have

In [10]:
#get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

In [11]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

In [15]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(float(total_missing)/float(total_cells)) * 100

24.87214126835169

In [19]:
# your turn! Find out what percent of the sf_permits dataset is missing
missing_count_sf = sf_permits.isnull().sum().sum()
total_cells_sf = np.product(sf_permits.shape)
float(missing_count_sf)/float(total_cells_sf)*100

26.26002315058403

### Figure out why the data is missing

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**
If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

In [21]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.


In [22]:
sf_permits.isnull().sum()

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

In [23]:
sf_permits.shape

(198900, 43)

In [26]:
#* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?
sf_permits[-sf_permits['Street Number Suffix'].isnull()].head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
92,201408254591,8,otc alterations permit,08/25/2014,3509,029,63,A,Grace,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,6.0,South of Market,94103.0,"(37.774944547030586, -122.41443674356925)",1353299454075
146,M803987,8,otc alterations permit,07/03/2017,0821,073,843,C,Hayes,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Hayes Valley,94117.0,"(37.775671880692904, -122.43022136710104)",1469150425694
164,201707141931,2,new construction wood frame,07/14/2017,1193,048,2150,A,Hayes,St,...,NaN,NaN,5.0,wood frame (5),Y,5.0,Lone Mountain/USF,94117.0,"(37.77347316353236, -122.45200535060522)",147059391960
200,M816647,8,otc alterations permit,08/04/2017,0821,073,843,C,Hayes,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Hayes Valley,94117.0,"(37.775671880692904, -122.43022136710104)",1473476425694
273,M829468,8,otc alterations permit,09/08/2017,3702,390,33,A,08th,St,...,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,94103.0,"(37.77850219390184, -122.413426065671)",1478111509834


In [ ]:
# `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets.
#I think both of them are not recorded

### Drop missing values

In [27]:
# remove all the rows that contain a missing value
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


It looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [29]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009


In [30]:
#how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 41


In [31]:
#Try removing all the rows from the sf_permits dataset that contain missing values. How many are left?
sf_permits.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


In [34]:
# Now try removing all the columns with empty values. Now how much of your data is left?
sf_permits_na_drop = sf_permits.dropna(axis=1)

In [36]:
print("the number of column in dropped na dataset is %d \n" % sf_permits_na_drop.shape[1] )
print("the number of column in original dataset is %d \n" % sf_permits.shape[1] )

the number of column in dropped na dataset is 12 

the number of column in original dataset is 43 



### Filling in missing values automatically

In [38]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


In [39]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [40]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [43]:
# Your turn! Try replacing all the NaN's in the sf_permits data with the one that
# comes directly after it and then replacing any remaining NaN's with 0
sf_permits.fillna(0).head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,Geary,St,...,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,0,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,0,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,0,Market,St,...,3.0,constr type 3,0.0,0,0,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
